In [7]:
import json
import numpy as np
from tqdm import tqdm
import os
import sys
sys.path.append("../")

import configparser
import psycopg2
import pymysql
import pymysql.cursors as pycursor
import numpy as np
import torch.optim as optim

import time
import glob
from models.concurrency.baselines.gcn_graph_gen import generate_graph
from models.concurrency.baselines.gcn_graph_embedding import load_data, accuracy, load_data_from_matrix, load_all_data
from models.concurrency.baselines.gcn import GCN
from models.concurrency.baselines.gcn_train import train_one_batch, eval_one_batch
from models.concurrency.baselines.gcn_constant import args
from models.concurrency.baselines.gcn import get_model, get_optimizer

In [2]:
data_dir = "/Users/ziniuw/Desktop/research/perf_prediction/workload-performance/pmodel_data/imdb/"
dataset = "sample-plan"
n_run_id = 4
print("Loading all data graphs")
all_adj, all_features, all_labels, all_root_idx_bit_map = load_all_data(data_dir, dataset, n_run_id)
n_graphs = len(all_adj)
print(f"In total, {n_graphs} number of graphs")

Loading all data graphs
In total, 835 number of graphs


In [11]:
all_features[0].shape

torch.Size([1078, 24])

In [12]:
model = get_model(feature_num=all_features[0].shape[-1], hidden=args.hidden, nclass=args.node_dim, dropout=args.dropout)
optimizer = get_optimizer(model=model, lr=args.lr, weight_decay=args.weight_decay)

In [13]:
all_idx = np.random.permutation(n_graphs)
train_idx = all_idx[0: int(0.85 * n_graphs)]
val_idx = all_idx[int(0.85 * n_graphs):]
best_val_loss = np.infty
num_epoch = 3
eval_every = 1
for epoch in range(num_epoch):
    model.train()
    curr_epoch_train_idx = np.random.permutation(train_idx)
    batch_loss = 0
    for i in tqdm(curr_epoch_train_idx):
        # per-batch training
        bl = train_one_batch(model, all_labels[i], all_features[i], all_adj[i], optimizer)
        batch_loss += bl
    if epoch % eval_every == 0:
        print(f"Epoch {epoch} ======================================================")
        print(f"Training loss: {batch_loss/len(curr_epoch_train_idx)}")
        model.eval()
        val_pred = []
        val_label = []
        val_loss = 0
        for i in val_idx:
            vl, pred, label = eval_one_batch(model, all_labels[i], all_features[i], all_adj[i],
                                             all_root_idx_bit_map[i])
            val_loss += vl
            val_pred.append(pred)
            val_label.append(label)
        val_pred = np.maximum(np.concatenate(val_pred), 1e-3)
        val_label = np.maximum(np.concatenate(val_label), 1e-3)
        print(f"Validation loss: {val_loss / len(val_idx)}")
        abs_error = np.abs(val_pred - val_label)
        q_error = np.maximum(val_pred / val_label, val_pred / val_label)
        for p in [50, 90, 95]:
            p_a = np.percentile(abs_error, p)
            p_q = np.percentile(q_error, p)
            print(f"{p}% absolute error is {p_a}, q-error is {p_q}")
        if val_loss < best_val_loss:
            best_val_loss = val_loss

100%|███████████████████████████████████████████████████████████████████████| 709/709 [00:00<00:00, 1335.21it/s]


Epoch 0 ======================================================
Training loss: 117.20987335011384
Validation loss: 112.44954018365769
50% absolute error is 4.2778990268707275, q-error is 876.9010009765625
90% absolute error is 4.278287410736084, q-error is 4279.27685546875
95% absolute error is 8.865517616271973, q-error is 4279.27734375


100%|███████████████████████████████████████████████████████████████████████| 709/709 [00:00<00:00, 1452.24it/s]


Epoch 1 ======================================================
Training loss: 205.44493497410008
Validation loss: 145.7657977058774
50% absolute error is 4.157362461090088, q-error is 852.1852416992188
90% absolute error is 4.15795931816101, q-error is 4158.66796875
95% absolute error is 8.986354684829712, q-error is 4158.67041015625


100%|███████████████████████████████████████████████████████████████████████| 709/709 [00:00<00:00, 1441.19it/s]


Epoch 2 ======================================================
Training loss: 145.1187259394292
Validation loss: 919.1502108649602
50% absolute error is 4.501829385757446, q-error is 922.96728515625
90% absolute error is 4.505601119995117, q-error is 4504.1787109375
95% absolute error is 8.656824874877927, q-error is 4504.191235351563


In [14]:
val_pred

array([4.504184 , 4.5041723, 4.5040274, ..., 4.5043116, 4.5041933,
       4.503839 ], dtype=float32)

In [15]:
val_label

array([1.0000000e-03, 1.0000000e-03, 2.4495367e+01, ..., 1.8260001e-03,
       1.0000000e-03, 1.1834924e+01], dtype=float32)

In [ ]:
data_path = '/Users/ziniuw/Desktop/research/perf_prediction/workload-performance/pmodel_data/job/'

edge_dim = 25000 # upper bound of edges
node_dim = 300 # upper bound of nodes

mp_optype = {'Aggregate': 0, 'Nested Loop': 1, 'Index Scan': 2, 'Hash Join': 3, 'Seq Scan': 4, 'Hash': 5, 'Update': 6} # operator types in the queries

oid = 0 # operator number

In [ ]:
class DictParser(configparser.ConfigParser):
    def read_dict(self):
        d = dict(self._sections)
        for k in d:
            d[k] = dict(d[k])
        return d

cf = DictParser()
cf.read("/Users/ziniuw/Desktop/research/perf_prediction/workload-performance/config.ini", encoding="utf-8")
config_dict = cf.read_dict()


def parse_knob_config(config_dict):
    _knob_config = config_dict["knob_config"]
    for key in _knob_config:
        _knob_config[key] = json.loads(str(_knob_config[key]).replace("\'", "\""))
    return _knob_config
knob_config = parse_knob_config(config_dict)

In [ ]:
knob_config

In [ ]:
"""
workloads = glob.glob(data_path + "sample-plan-*")

start_time = time.time()
num_graphs = 3000
for wid in range(num_graphs):
    st = time.time()

    vmatrix, ematrix, mergematrix = generate_graph(wid, connect=False)
    #print("[graph {}]".format(wid), "time:{}; #-vertex:{}, #-edge:{}".format(time.time() - st, len(vmatrix), len(ematrix)))
    
    with open(data_path + "graph/" + "sample-plan-" + str(wid) + ".content", "w") as wf:
        for v in vmatrix:
            wf.write(str(v[0]) + "\t" + str(v[1]) + "\t" + str(v[2]) + "\t" + str(v[3]) + "\t" + str(v[4]) + "\n")
    with open(data_path + "graph/" + "sample-plan-" + str(wid) + ".cites", "w") as wf:
        for e in ematrix:
            wf.write(str(e[0]) + "\t" + str(e[1]) + "\t" + str(e[2]) + "\n")
    #with open(data_path + "graph/" + "sample-plan-" + str(wid) + ".merges", "w") as wf:
     #   for e in mergematrix:
      #      wf.write(str(e[0]) + "\t" + str(e[1]) + "\t" + str(e[2]) + "\n")

end_time = time.time()

print("Total Time:{}".format(end_time - start_time))
"""

In [ ]:
workloads = glob.glob(save_dir + "sample-plan-0-*-nodes.npy")

In [ ]:
workloads

In [ ]:
save_dir = "/Users/ziniuw/Desktop/research/perf_prediction/workload-performance/pmodel_data/imdb/"
for wid in range(2):
    # Load data 
    adj, features, labels, root_idx_bit_map, idx_train, idx_val, idx_test = load_data(path = save_dir, dataset = f"sample-plan-0-{wid}" )
    print(adj.shape, features.shape, labels.shape, root_idx_bit_map.shape)

In [ ]:
labels[idx_val]

In [ ]:
output = model(features, adj)

In [ ]:
preds = output[idx_val].max(1)[1].type_as(labels)

In [ ]:
output.max(1)[0]

In [ ]:
preds

In [ ]:
model = GCN(nfeat=24,
            nhid=10,
            nclass=128, 
            dropout=0.1)    

optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
iteration_num = 3
num_epoch = 30
for wid in range(iteration_num):
    print("[graph {}]".format(wid))
    # Load data 
    adj, features, labels, root_idx_bit_map, idx_train, idx_val, idx_test = load_data(path = save_dir, dataset = f"sample-plan-0-{wid}")
    labels = labels.reshape(-1, 1)
    
    # Model Training
    ok_times = 0
    t_total = time.time()
    for epoch in range(num_epoch):
        # print(features.shape, adj.shape)
        if epoch == num_epoch - 1:
            debug = True
        else:
            debug = False
        loss_train = train(epoch, labels, features, adj, idx_train, idx_val, model, optimizer, debug)
        if loss_train < 0.002:
            ok_times += 1
        if ok_times >= 20:
            break    
        
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))